In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision

from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# 
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images/'
files = os.listdir(TEST_PATH)

In [ ]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
    #transforms.Scale(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

def default_loader(path):
    img_pil =  Image.open(path)
    img_pil = img_pil.resize((400,300))
    img_tensor = preprocess(img_pil)
    return img_tensor

In [ ]:
class MyTestDataset(Dataset):
    def __init__(self, files, loader=default_loader):
        self.files = files
        self.loader = loader
        
    def __getitem__(self, index):
        img_path = os.path.join(TEST_PATH, self.files[index])
        img = self.loader(img_path)
        
        return self.files[index], img
    
    def __len__(self):
        return len(self.labels)

In [ ]:
testset = MyTestDataset(files)

In [ ]:
MODEL_PATH = '../input/cassavaleaffsolmodels/resnet1.mdl'
model = torchvision.models.resnet34()
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

In [ ]:
labels = []
for file in files:
    img_path = os.path.join(TEST_PATH, file)
    img = default_loader(img_path)
    
    logits = model(img.unsqueeze(0))
    _, predicted_label = torch.max(logits, 1)
    labels.append(int(predicted_label))
submission = pd.DataFrame({'image_id':files, 'label':labels})

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index=False)